In [1]:
import matplotlib as mpl #画图用的库
import matplotlib.pyplot as plt
#下面这一句是为了可以在notebook中画图
%matplotlib inline
import numpy as np
import sklearn   #机器学习算法库
import pandas as pd #处理数据的库   
import os
import sys
import time
import tensorflow as tf
 
from tensorflow import keras   #使用tensorflow中的keras
#import keras #单纯的使用keras
 
print(tf.__version__)
print(sys.version_info)
for module in mpl, np, sklearn, pd, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.18.0
sklearn 0.21.3
pandas 0.25.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
loaded_keras_model = keras.models.load_model('./graph_def_and_weights/fashion_mnist_weights.h5')
loaded_keras_model(np.ones((1,28,28,1)))

<tf.Tensor: id=1762, shape=(1, 10), dtype=float32, numpy=
array([[3.4305863e-09, 3.6168969e-01, 6.5676955e-04, 1.2235546e-08,
        2.0280746e-13, 4.3944897e-08, 1.8053763e-13, 1.2191995e-06,
        1.2059320e-01, 5.1705909e-01]], dtype=float32)>

In [4]:
#keras model 保存为 TFLite
keras_to_tflite_converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
#量化tflite 模型，可以在损失较小精度或不影响精度的情况下减小模型大小
keras_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
keras_tflite=keras_to_tflite_converter.convert()

if not os.path.exists('./tflite_models'):
    os.mkdir('./tflite_models')
#with open('./tflite_models/keras_tflite', 'wb') as f:
#    f.write(keras_tflite)
#保存量化的 tflite 模型    
with open('./tflite_models/quantized_keras_tflite', 'wb') as f:
    f.write(keras_tflite)

In [5]:
run_model = tf.function(lambda x : loaded_keras_model(x))
keras_concrete_function = run_model.get_concrete_function(
    tf.TensorSpec(loaded_keras_model.inputs[0].shape,
                  loaded_keras_model.inputs[0].dtype)
)
keras_concrete_function(tf.constant(np.ones((1,28,28,1), dtype=np.float32)))

<tf.Tensor: id=2258, shape=(1, 10), dtype=float32, numpy=
array([[3.4305863e-09, 3.6168969e-01, 6.5676955e-04, 1.2235546e-08,
        2.0280746e-13, 4.3944897e-08, 1.8053763e-13, 1.2191995e-06,
        1.2059320e-01, 5.1705909e-01]], dtype=float32)>

In [6]:
#concrete function 保存为 TFLite
concrete_func_to_tflite_converter = tf.lite.TFLiteConverter.from_concrete_functions([keras_concrete_function])
#量化tflite 模型，可以在损失较小精度或不影响精度的情况下减小模型大小
concrete_func_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
concrete_func_tflite = concrete_func_to_tflite_converter.convert()

#with open('./tflite_models/concrete_func_tflite', 'wb') as f:
#    f.write(concrete_func_tflite)
#保存量化的 tflite 模型
with open('./tflite_models/quantized_concrete_func_tflite', 'wb') as f:
    f.write(concrete_func_tflite)

In [7]:
#savedModel 保存为 TFLite
saved_model_to_tflite_converter = tf.lite.TFLiteConverter.from_saved_model('./keras_saved_graph/')
#量化tflite 模型，可以在损失较小精度或不影响精度的情况下减小模型大小
saved_model_to_tflite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
saved_model_tflite = saved_model_to_tflite_converter.convert()

#with open('./tflite_models/saved_model_tflite', 'wb') as f:
#    f.write(saved_model_tflite)
#保存量化的 tflite 模型
with open('./tflite_models/quantized_saved_model_tflite', 'wb') as f:
    f.write(concrete_func_tflite)    

In [10]:
#
# tflite 解释器
#

#以前面的concrete_func_tflite为例
interpreter = tf.lite.Interpreter(model_content=concrete_func_tflite)
interpreter.allocate_tensors()#给所有的tensor分配内存

#获取 input 和 output tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#输出字典
print(input_details)
print(output_details)

[{'name': 'conv2d_input', 'index': 1, 'shape': array([ 1, 28, 28,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
[{'name': 'Identity', 'index': 0, 'shape': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [11]:
#手动设置输入参数
input_shape = input_details[0]['shape']#设置输入维度
input_data = tf.constant(np.ones(input_shape, dtype=np.float32))
interpreter.set_tensor(input_details[0]['index'],input_data)

#执行预测
interpreter.invoke()

output_results = interpreter.get_tensor(output_details[0]['index'])
print(output_results)#十个标签概率值的概率分布

[[2.3221240e-08 2.5767106e-01 1.1340029e-03 6.2593401e-08 2.9228692e-12
  3.0864484e-07 3.4217976e-12 6.7463984e-06 2.1669313e-01 5.2449471e-01]]
